This script is a programme that is used to determine SIR rates in the Bristol University first year population using an agent based modelling system. In order to run, the programme must be given initial conditions which include a text file of the names of all the people in the population and initial infection proportions for the top five most prevelent nationalities entering the university at the start of term.

The Script is comprised of 4 modules. The Main.py, Metrics.py, Functions.py and Assignments.py. Assignments.py initiates the Person class in a few steps. The first step is..........

LUDO WRITE ABOUT THE CODE YOU MADE FOR ASSIGNMENTS 

Metrics.py contains the functions which prepare the data for visualisation aswell as collecting the relevant data which the user wants to see. The main function in this module is CreateDataFrame(). This function does not take any arguments. It iterates through the attributes of the Person class and adds these as columns. Subsequently it adds the attribute data to a column list for each student. The Function then returns a pandas data frame of this data. 

In [3]:
import pandas as pd 
import random as r
import Assignments as s

# Function to create dataframe of students information  -- returns dataframe of all the students attributes 
def CreateDataFrame():
    
    # Creation of empty variable list to append column headings
    Columns = []

    # Create empty dictionary for values of each column for each student
    StudentsDict = {}

    # Iterates through the attributes of each student defined by the Person class and assigns an empty list to each one
    for keys, values in vars(s.StudentObjects[0]).items():
        Columns.append(keys)
        StudentsDict[keys] = []

    # Iterate through Student Numbers 
    for StudentNumber in range(s.PopulationSize):  

        # Extract student as an object 
        StudentInfo = s.StudentObjects[StudentNumber]

        # Extract student attributes iteratively and add to the list which is a value of the attribute key
        for column in Columns:
            value = getattr(StudentInfo, column)
            StudentsDict[column].append(value) 
            
     # Convert dictionary into a pandas dataframe 
    StudentInfoDataFrame = pd.DataFrame(StudentsDict) 
    
    return StudentInfoDataFrame

print(CreateDataFrame())
        

     StudentNumber  Halls Origin         Name SIR Course    Social  \
0                0   East     UK      Michael   S   STEM  3.591004   
1                1  South     EU  Christopher   S   STEM  3.432075   
2                2   West     UK      Jessica   S   STEM  3.987709   
3                3  North     UK      Matthew   S   ARTS  3.805069   
4                4  North     UK       Ashley   S   STEM  4.877696   
..             ...    ...    ...          ...  ..    ...       ...   
995            995  South     UK         Mike   S   STEM  4.275520   
996            996  North     UK        Chloe   S   STEM  4.978867   
997            997   East  China       Alecia   S   ARTS  3.844541   
998            998  South     UK          Sam   S   ARTS  1.495135   
999            999   West     UK        Rocio   S   ARTS  2.665871   

    DayInfected DayRecovered  
0        [0, 0]       [0, 0]  
1        [0, 0]       [0, 0]  
2        [0, 0]       [0, 0]  
3        [0, 0]       [0, 0]  
4   

Another very important function in the Metrics.py module is the CollectSIRdata(S,I,R) function. This uses the SIRinfoFilter function to collect all the people of the same SIR statuses and adds the proportion of the population they make up to a list to be used later in the Main data series for each status. In the Main.py, this function is called every day in order to collect the data on how the infection function for that day has affected the population. There is an easy option to collect the frequency aswell as a variable earlier on in the function that doesnt take into accoun the population size. 

In [5]:
# Function to collect SIR data for population -- returns a dataframe of proportiona and frequency against SIR 
def CollectSIRdata(S,I,R):
    
    # Uses SIRinfoFilter to find all the people of SIR values respectively 
    todaysFrequencyS = len(SIRinfoFilter('S', None, None, None, None, True,True))
    todaysFrequencyI = len(SIRinfoFilter('I', None, None, None, None, True,True))
    todaysFrequencyR = len(SIRinfoFilter('R', None, None, None, None, True,True))
    
    # Find Proportion
    todaysProportionS = todaysFrequencyS/s.PopulationSize
    todaysProportionI = todaysFrequencyI/s.PopulationSize
    todaysProportionR = todaysFrequencyR/s.PopulationSize
    
    proportionsSIR = [todaysProportionS,todaysProportionI,todaysProportionR]
    
    S.append(proportionsSIR[0])
    I.append(proportionsSIR[1])
    R.append(proportionsSIR[2])     

The most important and versatile function in the Metrics.py module is the SIRinfoFilter function. This function is reused throughout the code as it allows a user to return a dataframe containing all the agents of the population that have attributes specified as an argument in the function. Another feauture of the function is a variable which allows a conditional day for the function to be called aswell. 

In [ ]:
def SIRinfoFilter(SIRstatus, Origin, Name, Halls, Course, Day,CollectDataDay):
    
    if Day == CollectDataDay:
        
        # Reuses CreateDataframe function to produce a data frame of all students data
        StudentInfoDataFrame = CreateDataFrame()
        
        #   Checks to see if each respective attribute to be filtered by was defined when inputting the function 
        if isinstance(SIRstatus,str) == True:    
            
            # If it was defined then it adds the filtering condition tot the dataframe
            StudentInfoDataFrameFiltered = StudentInfoDataFrame[(StudentInfoDataFrame['SIR'] == SIRstatus)]   
        else:
            
            # If no infomation was given to filter then it does not filter it by the variable and says so to the user       
            StudentInfoDataFrameFiltered = StudentInfoDataFrame[~(StudentInfoDataFrame['SIR'] == SIRstatus)] 
            
        
        # These steps are repeated for each attribute that could be filtered by  
        if isinstance(Origin,str) == True:       
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[(StudentInfoDataFrameFiltered['Origin'] == Origin) ]
        else:        
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[~(StudentInfoDataFrameFiltered['Origin'] == Origin) ]
            
        
        if isinstance(Name,str) == True:       
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[(StudentInfoDataFrameFiltered['Name'] == Name) ]
        else:         
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[~(StudentInfoDataFrameFiltered['Name'] == Name) ]
            
        
        if isinstance(Halls,str) == True:
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[(StudentInfoDataFrameFiltered['Halls'] == Halls) ]
        else:         
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[~(StudentInfoDataFrameFiltered['Halls'] == Halls) ]
                 
            
        if isinstance(Course,str) == True:
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[(StudentInfoDataFrameFiltered['Course'] == Course) ]
        else:        
            StudentInfoDataFrameFiltered = StudentInfoDataFrameFiltered[~(StudentInfoDataFrameFiltered['Course'] == Course) ]
             
        
        # The filtered dataframe is returned
        return  StudentInfoDataFrameFiltered    
                         

The result of these functions and classes enable us to be able to manipulate a lot of data in quite a few ways. The following code is largely reused, with variable and position changes in order to show different plots, such as the distrbution of SIR quantities within different courses, halls and nationalitys.

In [ ]:
def BarChartCountries(Day, DateRange,CollectDataDay):
    
    if Day == CollectDataDay:
        
        countries = 'China' , 'US' , 'UK' , 'EU'

        SIRinputChoice = input('input S,I,R for SIR vals in y axis for countries plot:')
        
        if SIRinputChoice == 'S':
            yAxisName = "Susceptible"
        elif SIRinputChoice == 'I':
            yAxisName = "Infected"
        else:
           yAxisName = "Recovered"
        
        china = len(SIRinfoFilter(SIRinputChoice, 'China', None, None, None, Day,Day))
        us = len(SIRinfoFilter(SIRinputChoice, 'US', None, None, None, Day,Day))
        uk = len(SIRinfoFilter(SIRinputChoice, 'UK', None, None, None, Day,Day))
        eu = len(SIRinfoFilter(SIRinputChoice, 'EU', None, None, None, Day,Day))
        
        SIR = [china,us,uk,eu]
    
        plt.bar(countries,SIR)
        plt.title('Nationality infection data:'+ str(DateRange[Day]))
        plt.xlabel('Nationality')
        plt.ylabel(yAxisName)
        plt.show()
        df = (CreateDataFrame())
        MainData = pd.ExcelWriter('MainSpreadsheet.xlsx')
        df.to_excel(MainData)
        MainData.save()